In [4]:
import xgboost as xbg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
import joblib

%matplotlib inline
# comment 1

In [5]:
def load_cls(fname):
    return np.array(pickle.load(open(fname,"rb")))
# add path to .p file
path = "/scratch/pbanerj6/sml-dataset/"

X_train_cls = load_cls(path+"X_train_cls.p")
X_val_cls = load_cls(path+"X_val_cls.p")

In [6]:
y_train = load_cls(path+"y_train.p")
y_val = load_cls(path+"y_val.p")

In [7]:
from xgboost import XGBClassifier

model = XGBClassifier(n_jobs=-1,verbosity=2)

In [8]:
model.fit(X_train_cls,y_train,verbose=True,eval_set=[(X_val_cls,y_val)])

[0]	validation_0-error:0.4378
[1]	validation_0-error:0.4322
[2]	validation_0-error:0.429
[3]	validation_0-error:0.417933
[4]	validation_0-error:0.416933
[5]	validation_0-error:0.410267
[6]	validation_0-error:0.407267
[7]	validation_0-error:0.408467
[8]	validation_0-error:0.405267
[9]	validation_0-error:0.400933
[10]	validation_0-error:0.398467
[11]	validation_0-error:0.405867
[12]	validation_0-error:0.399733
[13]	validation_0-error:0.400067
[14]	validation_0-error:0.397467
[15]	validation_0-error:0.398533
[16]	validation_0-error:0.398533
[17]	validation_0-error:0.399667
[18]	validation_0-error:0.397933
[19]	validation_0-error:0.398133
[20]	validation_0-error:0.394933
[21]	validation_0-error:0.392133
[22]	validation_0-error:0.388867
[23]	validation_0-error:0.391867
[24]	validation_0-error:0.391667
[25]	validation_0-error:0.3908
[26]	validation_0-error:0.3898
[27]	validation_0-error:0.3882
[28]	validation_0-error:0.3858
[29]	validation_0-error:0.3858
[30]	validation_0-error:0.382
[31]	va

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1, verbosity=2)

In [9]:
from sklearn.metrics import accuracy_score

In [13]:
preds_val = model.predict(X_val_cls)

In [14]:
accuracy = accuracy_score(y_val, preds_val)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 62.48%


In [15]:
X_test_cls = load_cls(path+"X_test_cls.p")
y_test = load_cls(path+"y_test.p")

In [18]:
print(len(X_train_cls), len(X_test_cls), len(X_val_cls))

75000 15000 15000


In [16]:
preds_test = model.predict(X_test_cls)
accuracy = accuracy_score(y_test, preds_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 62.13%


In [40]:
X_test_cls[0].shape[0]

768

In [39]:
from scipy.stats import rankdata

def mrrs(out, labels):
#     print(out,labels)
    outputs = np.argmax(out,axis=1)
    mrr = 0.0 
    for label,ranks in zip(labels,out):
        ranks = rankdata(ranks*-1)
        rank = ranks[label]
#         print(rank,ranks)
        mrr+=1/rank
    return mrr/len(labels)

def mrrwrapper(qid2c,qid2indexmap,preds_prob):
    labels = []
    out = []
    for qid in qid2c.keys():
        scores = []
        for ix in qid2indexmap[qid]:
            if len(scores) < 6:
                scores.append(preds_prob[ix][1])
        if len(scores) < 6:
            continue
        out.append(scores)
        labels.append(int(qid2c[qid]))
    return mrrs(np.array(out),labels)

def load_ranking(fname):
    return pickle.load(open(path+"ranking_"+fname+".p","rb"))

In [41]:
preds_test_probs = model.predict_proba(X_test_cls)

In [42]:
qid2c,qid2indexmap = load_ranking("test")

In [43]:
mrrwrapper(qid2c,qid2indexmap,preds_test_probs)

0.5720777777777807